In [18]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix

In [19]:
df = pd.read_csv('dnd_monsters_simpler.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    float64
 2   type       401 non-null    object 
 3   size       401 non-null    object 
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      172 non-null    object 
 7   legendary  35 non-null     object 
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int64(2), object(5)
memory usage: 44.0+ KB


## Primeiro teste: Prevendo o CR (Challenge Rating) da criatura
### Convertendo os dados para numéricos (para que o KNN funcione)
O campo que mostra se é Lendário ou não: colocamos 1 caso seja e 0 caso não seja

Realizamos um encoder para criar índices numéricos para o type (tipo de criatura), size (categoria de tamanho) e speed (tipos de velocidades adicionais e.g.: fly, swim)

Como vamos prever o CR, o transformamos em categórico

No fim o CR possui várias categorias (de CR 0 até 30), isso causará problemas na hora da predição

In [20]:
df['legendary'] = df['legendary'].replace(['Legendary'], 1)
df['legendary'] = df['legendary'].fillna(0)

categorical_mappings = dict()
categorical_columns = ['type', 'size', 'speed']

for col in categorical_columns:
    le = LabelEncoder().fit(df[col])
    categorical_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    df[col] = le.transform(df[col])

print(categorical_mappings)

df.loc[df['cr'] == 0.125, 'cr'] = '1/8'
df.loc[df['cr'] == 0.25, 'cr'] = '1/4'
df.loc[df['cr'] == 0.5, 'cr'] = '1/2'

for i in range(len(df['cr'])):
    df['cr'][i] = str(df['cr'][i]).split('.')[0]

df['cr'].astype('object')

{'type': {'aberration': 0, 'beast': 1, 'celestial': 2, 'construct': 3, 'dragon': 4, 'elemental': 5, 'fey': 6, 'fiend': 7, 'giant': 8, 'humanoid': 9, 'monstrosity': 10, 'ooze': 11, 'plant': 12, 'swarm': 13, 'undead': 14}, 'size': {'Gargantuan': 0, 'Huge': 1, 'Large': 2, 'Medium': 3, 'Small': 4, 'Tiny': 5}, 'speed': {'fly': 0, 'fly, swim': 1, 'swim': 2, nan: 3}}


C:\Users\leona\AppData\Local\Temp\ipykernel_10368\2740291270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cr'][i] = str(df['cr'][i]).split('.')[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_10368\2740291270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cr'][i] = str(df['cr'][i]).split('.')[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_10368\2740291270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cr'

0      1/4
1       10
2      1/4
3       14
4       16
      ... 
396      3
397      4
398      1
399     26
400    1/4
Name: cr, Length: 401, dtype: object

In [21]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    object 
 2   type       401 non-null    int32  
 3   size       401 non-null    int32  
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      401 non-null    int32  
 7   legendary  401 non-null    float64
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int32(3), int64(2), object(2)
memory usage: 39.3+ KB


,name,cr,type,size,ac,hp,speed,legendary,str,dex,con,int,wis,cha
0,aarakocra,1/4,9,3,12,13,0,0.0,10.0,14.0,10.0,11.0,12.0,11.0
1,aboleth,10,0,2,17,135,2,1.0,21.0,9.0,15.0,18.0,15.0,18.0
2,acolyte,1/4,9,3,10,9,3,0.0,10.0,10.0,10.0,10.0,14.0,11.0
3,adult-black-dragon,14,4,1,19,195,1,1.0,23.0,14.0,21.0,14.0,13.0,17.0
4,adult-blue-dragon,16,4,1,19,225,0,1.0,25.0,10.0,23.0,16.0,15.0,19.0


## Preparando para a divisão de testes
Primeiro removemos a coluna name que não será usada para prever os dados

Segundo preparamos o X (sendo o dataframe sem o CR) e o Y (sendo a coluna CR)

Separamos em 70% de treino e 30% de teste

In [22]:
nameless_df = df.drop('name', axis=1)
X = nameless_df.drop('cr', axis=1)
y = nameless_df['cr']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

Testamos de 1 a 20 neighbors para ver qual performa melhor no KNN nesse caso em específico

Escolhemos o accuracy_score para ver qual foi o melhor, visto que o f1 e o precision_score só funcionam para resultados binários (duas categorias, como sim ou não) que não é o nosso caso nesse teste.

In [23]:
acc_scores = []
for i in range(20):
    knn = KNeighborsClassifier(n_neighbors=(i + 1))
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_scores.append(acc)
print(acc_scores)

[0.36363636363636365, 0.2727272727272727, 0.2975206611570248, 0.3140495867768595, 0.3305785123966942, 0.3140495867768595, 0.30578512396694213, 0.2727272727272727, 0.256198347107438, 0.2892561983471074, 0.30578512396694213, 0.2892561983471074, 0.2727272727272727, 0.2809917355371901, 0.2644628099173554, 0.2727272727272727, 0.2809917355371901, 0.2809917355371901, 0.2727272727272727, 0.256198347107438]


Podemos ver que nenhum score chegou a 40%, o que é péssimo.

Se fossemos escolher um valor puramente pelo seu tamanho, escolheríamos o primeiro valor.
Porém o primeiro significa que o número de neighbors seria 1, o que também não é legal, visto que ele só olharia o primeiro valor perto dele. 
Dito isso, acredito que foi pura sorte esse ser o primeiro colocado nos valores.

Um valor melhor para escolher seria o 5, já que foi o segundo maior, e 5 é bem mais em conta.

A matriz de confusão é simplesmente uma loucura para ler, visto que são muitas dimensões

In [24]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(acc)
print(cm)

0.3305785123966942
[[5 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 2 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 7 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 2 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 2 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 2 0 0

## Segundo teste: Prevendo o CR (Challenge Rating) da criatura, porém filtrando para menos opções
Nesse segundo teste, nós tentamos diminuir o range de 0 a 30, para apenas 4 valores: Fácil, Médio, Difícil, Mortal, usando o seguinte critério para a conversão:

 - Fácil: 5 ou menos
 - Médio: 6 - 12
 - Difícil: 13 - 20
 - Mortal: 21 ou mais

### Convertendo os dados para numéricos (para que o KNN funcione)
O campo que mostra se é Lendário ou não: colocamos 1 caso seja e 0 caso não seja

Realizamos um encoder para criar índices numéricos para o type (tipo de criatura), size (categoria de tamanho) e speed (tipos de velocidades adicionais e.g.: fly, swim)

E realizamos a conversão de CR

In [25]:
df = pd.read_csv('dnd_monsters_simpler.csv')

df['legendary'] = df['legendary'].replace(['Legendary'], 1)
df['legendary'] = df['legendary'].fillna(0)

categorical_mappings = dict()
categorical_columns = ['type', 'size', 'speed']

for col in categorical_columns:
    le = LabelEncoder().fit(df[col])
    categorical_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    df[col] = le.transform(df[col])

print(categorical_mappings)

def convert_cr(cr):
    if float(cr) <= 5:
        return 'Fácil'
    if 5 < float(cr) <= 12:
        return 'Médio'
    if 12 < float(cr) <= 20:
        return 'Difícil'
    if float(cr) > 20:
        return 'Mortal'

df['cr'] = df['cr'].apply(convert_cr)

df['cr'].astype('object')
df['cr'].value_counts()

{'type': {'aberration': 0, 'beast': 1, 'celestial': 2, 'construct': 3, 'dragon': 4, 'elemental': 5, 'fey': 6, 'fiend': 7, 'giant': 8, 'humanoid': 9, 'monstrosity': 10, 'ooze': 11, 'plant': 12, 'swarm': 13, 'undead': 14}, 'size': {'Gargantuan': 0, 'Huge': 1, 'Large': 2, 'Medium': 3, 'Small': 4, 'Tiny': 5}, 'speed': {'fly': 0, 'fly, swim': 1, 'swim': 2, nan: 3}}


cr
Fácil      291
Médio       63
Difícil     31
Mortal      16
Name: count, dtype: int64

In [26]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    object 
 2   type       401 non-null    int32  
 3   size       401 non-null    int32  
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      401 non-null    int32  
 7   legendary  401 non-null    float64
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int32(3), int64(2), object(2)
memory usage: 39.3+ KB


,name,cr,type,size,ac,hp,speed,legendary,str,dex,con,int,wis,cha
0,aarakocra,Fácil,9,3,12,13,0,0.0,10.0,14.0,10.0,11.0,12.0,11.0
1,aboleth,Médio,0,2,17,135,2,1.0,21.0,9.0,15.0,18.0,15.0,18.0
2,acolyte,Fácil,9,3,10,9,3,0.0,10.0,10.0,10.0,10.0,14.0,11.0
3,adult-black-dragon,Difícil,4,1,19,195,1,1.0,23.0,14.0,21.0,14.0,13.0,17.0
4,adult-blue-dragon,Difícil,4,1,19,225,0,1.0,25.0,10.0,23.0,16.0,15.0,19.0


## Preparando para a divisão de testes
Primeiro removemos a coluna name que não será usada para prever os dados

Segundo preparamos o X (sendo o dataframe sem o CR) e o Y (sendo a coluna CR)

Separamos em 70% de treino e 30% de teste

In [27]:
nameless_df = df.drop('name', axis=1)
X = nameless_df.drop('cr', axis=1)
y = nameless_df['cr']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

Testamos de 1 a 20 neighbors para ver qual performa melhor no KNN nesse caso em específico

Escolhemos o accuracy_score para ver qual foi o melhor, visto que o f1 e o precision_score só funcionam para resultados binários (duas categorias, como sim ou não) que não é o nosso caso nesse teste.

In [28]:
acc_scores = []
for i in range(20):
    knn = KNeighborsClassifier(n_neighbors=(i + 1))
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    acc_scores.append(acc)
print(acc_scores)

[0.8016528925619835, 0.7933884297520661, 0.8099173553719008, 0.8099173553719008, 0.8181818181818182, 0.8016528925619835, 0.8347107438016529, 0.8429752066115702, 0.8347107438016529, 0.8347107438016529, 0.8347107438016529, 0.8264462809917356, 0.8264462809917356, 0.8347107438016529, 0.8099173553719008, 0.8347107438016529, 0.8016528925619835, 0.8016528925619835, 0.7933884297520661, 0.7933884297520661]


Comparado a antes, a acurácia chegou em 80%, o que é muito melhor que antes.

Nesse caso, o melhor score a ser escolhido é o com 8 neighbors, que obteve 84%.

In [29]:
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(acc)
print(cm)

0.8429752066115702
[[ 8  0  0  2]
 [ 0 74  0  6]
 [ 3  0  4  0]
 [ 2  6  0 16]]


A Matriz de confusão é bem mais legível dessa vez. A matriz está bem melhor em geral também.

Ainda existem erros.

Nossa interpretação sobre os erros é que como o dataframe possui apenas dados como atributos, HP (Pontos de Vida), AC (Classe de Armadura), Deslocamento de Voo ou Nado, mas não possui dados sobre as ações, ações bônus, reações, magias, ações lendárias, dano, saving throws (testes de resistência), algumas criaturas que possuem pouco nos valores que estão no dataframe, mas possuem muito dano ou ações perigosas foram previstas como Fácil, visto que não há informações sobre essas ações, dano, etc.

## Terceiro teste: Prevendo se uma criatura é Lendária ou não
Vamos ao último teste. Vamos fazer tudo que fizemos novamente, porém não mexeremos no CR e vamos usar o y como Legendary.

Diferente da última vez, colocamos o Legendary como Yes e No.

In [30]:
df = pd.read_csv('dnd_monsters_simpler.csv')

df['legendary'] = df['legendary'].replace(['Legendary'], 'Yes')
df['legendary'] = df['legendary'].fillna('No')

categorical_mappings = dict()
categorical_columns = ['type', 'size', 'speed']

for col in categorical_columns:
    le = LabelEncoder().fit(df[col])
    categorical_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    df[col] = le.transform(df[col])

print(categorical_mappings)

{'type': {'aberration': 0, 'beast': 1, 'celestial': 2, 'construct': 3, 'dragon': 4, 'elemental': 5, 'fey': 6, 'fiend': 7, 'giant': 8, 'humanoid': 9, 'monstrosity': 10, 'ooze': 11, 'plant': 12, 'swarm': 13, 'undead': 14}, 'size': {'Gargantuan': 0, 'Huge': 1, 'Large': 2, 'Medium': 3, 'Small': 4, 'Tiny': 5}, 'speed': {'fly': 0, 'fly, swim': 1, 'swim': 2, nan: 3}}


In [31]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    float64
 2   type       401 non-null    int32  
 3   size       401 non-null    int32  
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      401 non-null    int32  
 7   legendary  401 non-null    object 
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int32(3), int64(2), object(2)
memory usage: 39.3+ KB


,name,cr,type,size,ac,hp,speed,legendary,str,dex,con,int,wis,cha
0,aarakocra,0.25,9,3,12,13,0,No,10.0,14.0,10.0,11.0,12.0,11.0
1,aboleth,10.00,0,2,17,135,2,Yes,21.0,9.0,15.0,18.0,15.0,18.0
2,acolyte,0.25,9,3,10,9,3,No,10.0,10.0,10.0,10.0,14.0,11.0
3,adult-black-dragon,14.00,4,1,19,195,1,Yes,23.0,14.0,21.0,14.0,13.0,17.0
4,adult-blue-dragon,16.00,4,1,19,225,0,Yes,25.0,10.0,23.0,16.0,15.0,19.0


In [32]:
nameless_df = df.drop('name', axis=1)
X = nameless_df.drop('legendary', axis=1)
y = nameless_df['legendary']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

Nos scores, pudemos usar o precision e o f1

In [33]:
acc_scores = []
prc_scores = []
f1_scores = []
for i in range(20):
    knn = KNeighborsClassifier(n_neighbors=(i + 1))
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc_scores.append(accuracy_score(y_test, y_pred))
    prc_scores.append(precision_score(y_test, y_pred, average="binary", pos_label="No"))
    f1_scores.append(f1_score(y_test, y_pred, average="binary", pos_label="No"))
print(acc_scores)
print(prc_scores)
print(f1_scores)

[0.9338842975206612, 0.9173553719008265, 0.9338842975206612, 0.9090909090909091, 0.9256198347107438, 0.9338842975206612, 0.9421487603305785, 0.9421487603305785, 0.9338842975206612, 0.9338842975206612, 0.9338842975206612, 0.9338842975206612, 0.9338842975206612, 0.9338842975206612, 0.9338842975206612, 0.9421487603305785, 0.9421487603305785, 0.9421487603305785, 0.9421487603305785, 0.9504132231404959]
[0.9464285714285714, 0.9224137931034483, 0.9629629629629629, 0.9369369369369369, 0.9626168224299065, 0.9545454545454546, 0.963302752293578, 0.954954954954955, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.954954954954955, 0.954954954954955, 0.954954954954955, 0.954954954954955, 0.9473684210526315]
[0.9636363636363636, 0.9553571428571429, 0.9629629629629629, 0.9497716894977168, 0.958139534883721, 0.963302752293578, 0.9677419354838711, 0.9680365296803655, 0.963302752293578, 0.963302752293578, 0.96330

Os scores em si estão muito melhores dessa vez, já que o atributo previsto é binário.

Embora todas variações no número de neighbors sejam parecidas, decidimos escolher o número de neighbors como 7, visto que possui valores um pouco melhores, e não é tão grande.

In [34]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prc = precision_score(y_test, y_pred, average="binary", pos_label="No")
f1 = f1_score(y_test, y_pred, average="binary", pos_label="No")

cm = confusion_matrix(y_test, y_pred)
print(acc)
print(prc)
print(f1)
print(cm)

0.9421487603305785
0.963302752293578
0.9677419354838711
[[105   3]
 [  4   9]]


A matriz de confusão mostra que apenas 7 casos foram previstos erroneamente. Nessa base de dados há pouquíssimos casos em que uma criatura é lendária, portanto, pode ter sido difícil prever por este motivo também.